In [9]:
# Q1: Select top purchasing customer for each store based on total purchase amount
# ctrl+/ or ctrl+shift+/ to comment all selected lines
# Columns: store_id, customer_id, purchase_amt
# Table: sales

# Sample Input:
# store_id, customer_id, purchase_amt
# 1, 101, 500
# 1, 101, 300
# 1, 102, 500
# 2, 101, 1000
# 2, 102, 900
# 3, 101, 1000
# 3, 102, 1000

# Expected Output:
# 1, 101, 800
# 2, 101, 1000
# 3, 101, 1000
# 3, 102, 1000

# PySpark Solution:
from pyspark.sql import Window
from pyspark.sql.functions import col, sum as _sum, dense_rank
from pyspark.sql import SparkSession, Row

# Step 1: Aggregate total purchase amount per customer per store
# Create Spark session
spark = SparkSession.builder.appName("TopPurchasingCustomer").getOrCreate()

# Create sample data as per the input
data = [
    Row(store_id=1, customer_id=101, purchase_amt=500),
    Row(store_id=1, customer_id=101, purchase_amt=300),
    Row(store_id=1, customer_id=102, purchase_amt=500),
    Row(store_id=2, customer_id=101, purchase_amt=1000),
    Row(store_id=2, customer_id=102, purchase_amt=900),
    Row(store_id=3, customer_id=101, purchase_amt=1000),
    Row(store_id=3, customer_id=102, purchase_amt=1000),
]

# Create DataFrame
sales_df = spark.createDataFrame(data)

agg_df = sales_df.groupBy("store_id", "customer_id").agg(_sum("purchase_amt").alias("purchase_amt"))

# Step 2: Define window to rank customers by purchase_amt within each store
window_spec = Window.partitionBy("store_id").orderBy(col("purchase_amt").desc())

# Step 3: Add rank column
ranked_df = agg_df.withColumn("rnk", dense_rank().over(window_spec))

# Step 4: Filter to get top purchasing customers per store (can be ties)
result_df = ranked_df.filter(col("rnk") == 1).select("store_id", "customer_id", "purchase_amt")
result_df.show()

# Oracle SQL Solution (for reference):
# You can run this query using cx_Oracle after setting up your connection.

import cx_Oracle
import sys

# Database connection details
dsn_tns = cx_Oracle.makedsn('localhost', 1521, service_name='orcl')
try:
    conn = cx_Oracle.connect(user='Sivaacademy', password='securecode', dsn=dsn_tns)
    cursor = conn.cursor()

    # Check Oracle version
    cursor.execute("SELECT * FROM v$version WHERE banner LIKE 'Oracle%'")
    version = cursor.fetchone()
    print(f"Oracle Version: {version[0]}")

    # Create global temporary table (compatible with Oracle 12c and later)
    cursor.execute(""" DROP TABLE sales_temp PURGE""")
    
    cursor.execute("""
        CREATE GLOBAL TEMPORARY TABLE sales_temp (
            store_id NUMBER,
            customer_id NUMBER,
            purchase_amt NUMBER
        ) ON COMMIT PRESERVE ROWS
    """)

    # Bulk insert sample data
    sample_data = [
        (1, 101, 500),
        (1, 101, 300),
        (1, 102, 500),
        (2, 101, 1000),
        (2, 102, 900),
        (3, 101, 1000),
        (3, 102, 1000)
    ]
    cursor.executemany("""
        INSERT INTO sales_temp (store_id, customer_id, purchase_amt)
        VALUES (:1, :2, :3)
    """, sample_data)

    # Commit inserts
    conn.commit()

    # Execute query
    oracle_query = """
        WITH cte AS (
            SELECT 
                store_id, 
                customer_id, 
                SUM(purchase_amt) AS purchase_amt, 
                DENSE_RANK() OVER (PARTITION BY store_id ORDER BY SUM(purchase_amt) DESC) AS rnk 
            FROM sales_temp 
            GROUP BY store_id, customer_id
        )
        SELECT store_id, customer_id, purchase_amt 
        FROM cte 
        WHERE rnk = 1
    """
    cursor.execute(oracle_query)

    # Fetch and print results
    for row in cursor:
        print(row)

except cx_Oracle.DatabaseError as e:
    error, = e.args
    print(f"Oracle Error {error.code}: {error.message}")
    sys.exit(1)
except Exception as e:
    print(f"An error occurred: {e}")
    sys.exit(1)
finally:
    cursor.close()
    conn.close()

+--------+-----------+------------+
|store_id|customer_id|purchase_amt|
+--------+-----------+------------+
|       1|        101|         800|
|       2|        101|        1000|
|       3|        101|        1000|
|       3|        102|        1000|
+--------+-----------+------------+

Oracle Version: Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production
(1, 101, 800)
(2, 101, 1000)
(3, 101, 1000)
(3, 102, 1000)


In [ ]:
#Q2 : Read a file and extract the fifth word from the fifth line
# Read fifth line and extract the fifth word
# And print the length of the fifth word and the word itself
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Text_File_Reading").getOrCreate()

with open("D:/Data_Engineer/DE_Interview/input.txt", "r") as file:
    lines = file.readlines()
#Check if file has at least 5 lines - len() to get count of lines
if len(lines) >= 5:
    #strip() removes whitespace, split() converts string to list of words
    fifth_line = lines[4].strip()
    words = fifth_line.split()
    #Check if fifth line has at least 5 words - len() for word count
    if len(words) >= 5:
        fifth_word = words[4]
        print(f"Fifth word: {fifth_word}")
        print(f"Length of fifth word: {len(fifth_word)}")
    else:
        print("Less than 5 words in line 5")
else:
    print("Less than 5 lines in the file")

# Rewrite above code in pyspark Read the text file into a DataFrame
# Read fifth line and extract the fifth word
# And print the length of the fifth word and the word itself
# Read text file into a DataFrame; each line becomes a row with a single column 'value' (string)
text_df = spark.read.text("D:/Data_Engineer/DE_Interview/input.txt")
#Select 'value' column (the text of each line), limit to first 5 rows, and collect results to driver
# - spark.read.text: Creates a DataFrame where each row is a line from the file, stored in 'value' column
# - select("value"): Picks only the 'value' column (text of each line)
# - limit(5): Restricts DataFrame to first 5 rows (lines)
# - collect(): Action that retrieves all rows as a list of Row objects to the driver (local Python)
five_line_df = text_df.select("value").limit(5).collect()

# Check if at least 5 lines exist in the collected data
# - len(five_line_df): Counts rows in the collected list (number of lines)
# - Python logic since collect() brings data to driver
if len(five_line_df) >= 5:
    # Extract the 5th line's text (index 4, as lists are 0-based) from the 'value' column
    # - five_line_df[4]: 5th Row object
    # - ["value"]: Accesses the string in the 'value' column
    fifth_line = five_line_df[4]["value"]
    
    # Split the 5th line into words using whitespace as delimiter
    # - split(): Python string method, creates list of words
    words = fifth_line.split()
    
    # Check if the 5th line has at least 5 words
    # - len(words): Counts words in the split list
    if len(words) >= 5:
        # Get the 5th word (index 4, 0-based)
        fifth_word = words[4]
        
        # Print the 5th word
        print(f"Fifth word: {fifth_word}")
        
        # Print the length of the 5th word
        # - len(fifth_word): Python string length
        print(f"Length of fifth word: {len(fifth_word)}")
    else:
        # Handle case where 5th line has fewer than 5 words
        print("Less than 5 words in line 5")
else:
    # Handle case where file has fewer than 5 lines
    print("Less than 5 lines in the file")


Fifth word: words
Length of fifth word: 5
Fifth word: words
Length of fifth word: 5


In [ ]:
#Q3. Create a pyspark program to create a json file containing Person information like first_name, last_name, age, mobile, email, address(contains street, city, state, country) and write it to a file. And handle all kinds of interview question on json file like read, write, update, delete, filter, sort, etc. write the file and read the file from the same location.
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col, to_json, struct, concat_ws

# Create Spark session
spark = SparkSession.builder.appName("JSON Personal_data write and read ").getOrCreate()



# Define schema for Person information
# schema = StructType([
#     StructField("name", StringType(), True, {"description": "person's name", "example": "Alice"})
# ])  True is nullable, False is not nullable, {"description": "person's name", "example": "Alice"}} is metadata

person_schema = StructType([
    StructField("first_name", StringType(), False),
    StructField("last_name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("mobile", StringType(), True),
    StructField("email", StringType(), True),
    StructField("address", StructType([
        StructField("street", StringType(), True),
        StructField("city", StringType(), True),
        StructField("state", StringType(), True),
        StructField("country", StringType(), True)
    ]), True)
])

# | PySpark Data Type               | Python Equivalent          | Example               |
# | ------------------------------- | -------------------------- | --------------------- |
# | `StringType()`                  | `str`                      | `"John"`              |
# | `IntegerType()`                 | `int`                      | `25`                  |
# | `LongType()`                    | `int` (big integers)       | `9876543210`          |
# | `FloatType()`                   | `float`                    | `3.14`                |
# | `DoubleType()`                  | `float` (double-precision) | `2.71828`             |
# | `BooleanType()`                 | `bool`                     | `True`, `False`       |
# | `DateType()`                    | `datetime.date`            | `2023-01-01`          |
# | `TimestampType()`               | `datetime.datetime`        | `2023-01-01 10:00:00` |
# | `BinaryType()`                  | `bytes`                    | Binary data           |
# | `DecimalType(precision, scale)` | `decimal.Decimal`          | `Decimal("123.45")`   |


# Create sample data
data = [
    ("John", "Doe", 30, "1234567890", "john.doe@email.com",
     {"street": "123 Main St", "city": "New York", "state": "NY", "country": "USA"}),
    ("Jane", "Smith", 28, "9876543210", "jane.smith@email.com",
     {"street": "456 Oak Ave", "city": "Los Angeles", "state": "CA", "country": "USA"}),
    ("Michael", "Johnson", 35, "5551234567", "michael.j@email.com",
     {"street": "789 Pine Rd", "city": "Chicago", "state": "IL", "country": "USA"}),
    ("Sarah", "Williams", 26, "4567891230", "sarah.w@email.com",
     {"street": "321 Elm St", "city": "Houston", "state": "TX", "country": "USA"}),
    ("Robert", "Brown", 32, "7894561230", "robert.b@email.com",
     {"street": "654 Maple Dr", "city": "Phoenix", "state": "AZ", "country": "USA"}),
    ("Emily", "Davis", 29, "3216549870", "emily.d@email.com",
     {"street": "987 Cedar Ln", "city": "Miami", "state": "FL", "country": "USA"}),
    ("David", "Miller", 31, "1597534560", "david.m@email.com",
     {"street": "147 Birch Rd", "city": "Seattle", "state": "WA", "country": "USA"}),
    ("Lisa", "Wilson", 27, "3579514260", "lisa.w@email.com",
     {"street": "258 Spruce Ave", "city": "Denver", "state": "CO", "country": "USA"}),
    ("James", "Taylor", 33, "7531598520", "james.t@email.com",
     {"street": "369 Palm St", "city": "Boston", "state": "MA", "country": "USA"}),
    ("Amanda", "Anderson", 34, "9517538520", "amanda.a@email.com",
     {"street": "741 Beach Rd", "city": "San Diego", "state": "CA", "country": "USA"})
]

# Create DataFrame
person_df = spark.createDataFrame(data, schema=person_schema)

# Write to JSON file
person_df.write.mode("overwrite").json("D:/Data_Engineer/DE_Interview/person_data.json")

# Read JSON file
df_read = spark.read.json("D:/Data_Engineer/DE_Interview/person_data.json")

# Example operations:
# 1. Filter by age
filtered_df = df_read.filter(col("age") > 30)

# 2. Sort by name
sorted_df = df_read.orderBy("first_name")

# 3. Update - Add full name column
updated_df = df_read.withColumn("full_name", 
    concat_ws(" ", col("first_name"), col("last_name")))

# 4. Select specific fields
selected_df = df_read.select("first_name", "email", "address.city")

# Display results
df_read.show(truncate=False)
filtered_df.show(truncate=False)
sorted_df.show(truncate=False)
updated_df.show(truncate=False)
selected_df.show(truncate=False)


+-----------------------------------+---+--------------------+----------+---------+----------+
|address                            |age|email               |first_name|last_name|mobile    |
+-----------------------------------+---+--------------------+----------+---------+----------+
|{San Diego, USA, CA, 741 Beach Rd} |34 |amanda.a@email.com  |Amanda    |Anderson |9517538520|
|{Los Angeles, USA, CA, 456 Oak Ave}|28 |jane.smith@email.com|Jane      |Smith    |9876543210|
|{Chicago, USA, IL, 789 Pine Rd}    |35 |michael.j@email.com |Michael   |Johnson  |5551234567|
|{Phoenix, USA, AZ, 654 Maple Dr}   |32 |robert.b@email.com  |Robert    |Brown    |7894561230|
|{Houston, USA, TX, 321 Elm St}     |26 |sarah.w@email.com   |Sarah     |Williams |4567891230|
|{Seattle, USA, WA, 147 Birch Rd}   |31 |david.m@email.com   |David     |Miller   |1597534560|
|{Denver, USA, CO, 258 Spruce Ave}  |27 |lisa.w@email.com    |Lisa      |Wilson   |3579514260|
|{Boston, USA, MA, 369 Palm St}     |33 |james.t@e

In [3]:
# Q4. Create a pyspark program to create a json file containing employee information like emp_id, first_name, last_name, DOB, email, job_title, hire_date, salary, mgr_id, deptid and write it to a file. And handle all kinds of interview question on json file like read, write, update, delete, filter, sort, etc. write the file and read the file from the same location.

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
from pyspark.sql.functions import col, count,max, avg, sum, when, datediff,round, current_date, concat_ws
from pyspark.sql import SparkSession
from datetime import datetime
spark = SparkSession.builder.appName("JSON emp dept data write and read").getOrCreate()

# Create sample data as per the input
#data = [
#    Row(store_id=1, customer_id=101, purchase_amt=500)] # This way we doing when data is less and here we need to mention column names and its value
#sales_df = spark.createDataFrame(data) #directly creating dataframe from row object


emp_schema = StructType([
    StructField("emp_id", IntegerType(), False),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("DOB", DateType(), True),
    StructField("email", StringType(), True),
    StructField("job_title", StringType(), True),
    StructField("hire_date", DateType(), True),
    StructField("salary", IntegerType(), True),
    StructField("mgr_id", IntegerType(), True),
    StructField("dept_id", IntegerType(), True)
])

# Create dept_schema for Department information
dept_schema = StructType([
    StructField("dept_id", IntegerType(), False),
    StructField("dept_name", StringType(), True),
    StructField("location", StringType(), True)
])

emp_data = [
    (1, "Ramesh", "Singh", datetime(1995, 1, 5), "rameshsingh@gmail.com", "Data_Engineer", datetime(2020, 1, 5), 500000, 2, 1),
    (2, "Suresh", "Kumar", datetime(1992, 3, 15), "suresh@gmail.com", "Senior Engineer", datetime(2018, 6, 10), 600000, 3, 1),
    (3, "Priya", "Sharma", datetime(1990, 7, 20), "priya@gmail.com", "Manager", datetime(2015, 2, 15), 800000, None, 2),
    (4, "John", "Doe", datetime(1988, 12, 10), "john@gmail.com", "Developer", datetime(2019, 3, 20), 550000, 2, 3),
    (5, "Sarah", "Miller", datetime(1993, 5, 25), None, "Data Analyst", datetime(2021, 1, 15), 480000, 3, 1),
    (6, "Mike", "Wilson", datetime(1991, 9, 30), "mike@gmail.com", "Senior Manager", datetime(2016, 7, 1), 750000, None, 2),
    (7, "Emma", "Brown", datetime(1994, 4, 12), "emma@gmail.com", "Engineer", datetime(2020, 11, 30), 520000, 2, 1),
    (1, "Ramesh", "Singh", datetime(1995, 1, 5), "rameshsingh@gmail.com", "Data_Engineer", datetime(2020, 1, 5), 500000, 2, 1),  # Duplicate
    (8, None, "Taylor", datetime(1989, 8, 18), "taylor@gmail.com", "Architect", datetime(2017, 9, 15), 680000, 3, 3),
    (9, "Lisa", "Anderson", datetime(1992, 11, 22), "lisa@gmail.com", "Developer", datetime(2019, 5, 20), 540000, 2, 1),
    (10, "David", "Clark", None, "david@gmail.com", "Engineer", datetime(2021, 3, 10), 510000, 2, 2),
    (11, "Amy", "Wright", datetime(1993, 2, 28), "amy@gmail.com", "Data Analyst", datetime(2020, 8, 1), 490000, 3, 3),
    (12, "Tom", "Harris", datetime(1990, 6, 15), "tom@gmail.com", None, datetime(2018, 4, 25), 580000, 2, 1),
    (13, "Jessica", "Lee", datetime(1991, 12, 5), "jessica@gmail.com", "Senior Developer", datetime(2017, 11, 12), 650000, 3, 2),
    (6, "Mike", "Wilson", datetime(1991, 9, 30), "mike@gmail.com", "Senior Manager", datetime(2016, 7, 1), 750000, None, 2)  # Duplicate
]

dept_data = [
    (1, "IT", "Bangalore"),
    (2, "HR", "Mumbai"),
    (3, "Finance", "Delhi"),
    (4, "Marketing", None),
    (5, "Operations", "Chennai")
]

# Create DataFrames and clean data 
emp_df = spark.createDataFrame(emp_data, schema=emp_schema)
dept_df = spark.createDataFrame(dept_data, schema=dept_schema)

emp_df.write.mode("overwrite").json("D:/Data_Engineer/DE_Interview/emp_data.json")
dept_df.write.mode("overwrite").json("D:/Data_Engineer/DE_Interview/dept_data.json")

# Remove duplicates from emp_df based on emp_id and remove null values
clean_df = emp_df.dropDuplicates(["emp_id"]).filter(
    col("first_name").isNotNull() & 
    col("last_name").isNotNull() & 
    col("DOB").isNotNull() & 
    col("job_title").isNotNull() & 
    col("hire_date").isNotNull() & 
    col("salary").isNotNull() & 
    col("dept_id").isNotNull()
)

#Derived Columns
derived_df = clean_df.withColumn("Full_Name", concat_ws(" ", col("first_name"), col("last_name")))\
    .withColumn("age", round((datediff(current_date(), col("DOB"))/365.25),2))\
    .withColumn("YOE", round((datediff(current_date(), col("hire_date"))/365.25),2))
# Analysis Questions:
# 1. Find highest salary department wise and dept_name and emp_name
# First get max salary per department
max_salaries = derived_df.join(
    dept_df,
    derived_df.dept_id == dept_df.dept_id
).groupBy("dept_name").agg(
    max("salary").alias("highest_salary")
)

# 2. Find the total salary department wise dept_name, total_salary and count number of employee department wise
dept_salary = derived_df.join(
    dept_df,
    derived_df.dept_id == dept_df.dept_id
).groupBy("dept_name").agg(
    sum("salary").alias("total_salary"),
    count("emp_id").alias("employee_count")
)

# 3. List out the employee having experience more than 5 years
experienced_emp = derived_df.filter(col("YOE") > 5).select(
    "Full_Name",
    "job_title",
    "YOE",
    "salary"
).orderBy("YOE", ascending=False)

# 4. Department-wise average age of employees
dept_age = derived_df.join(
    dept_df,
    derived_df.dept_id == dept_df.dept_id
).groupBy("dept_name").agg(
    avg("age").alias("avg_age")
)

# 5. Department-wise salary distribution
salary_distribution = derived_df.join(
    dept_df,
    derived_df.dept_id == dept_df.dept_id
).select(
    "dept_name","emp_id", "Full_Name", "salary",
    when(col("salary") <= 500000, "Entry")
    .when(col("salary") <= 650000, "Mid")
    .otherwise("Senior").alias("salary_band")
)

# 6. Employees reporting to each manager
manager_subordinates = derived_df.alias("emp").join(
    derived_df.select("emp_id", "first_name").withColumnRenamed("first_name", "manager_name").alias("mgr"),
    col("emp.mgr_id") == col("mgr.emp_id"),
    "left"
).groupBy("manager_name").count()


# Display additional results
max_salaries.show()
dept_salary.show()
experienced_emp.show()
dept_age.show()
salary_distribution.show()
manager_subordinates.show()



+---------+--------------+
|dept_name|highest_salary|
+---------+--------------+
|       HR|        800000|
|  Finance|        550000|
|       IT|        600000|
+---------+--------------+

+---------+------------+--------------+
|dept_name|total_salary|employee_count|
+---------+------------+--------------+
|       HR|     2200000|             3|
|  Finance|     1040000|             2|
|       IT|     2640000|             5|
+---------+------------+--------------+

+-------------+----------------+-----+------+
|    Full_Name|       job_title|  YOE|salary|
+-------------+----------------+-----+------+
| Priya Sharma|         Manager|10.29|800000|
|  Mike Wilson|  Senior Manager| 8.92|750000|
|  Jessica Lee|Senior Developer| 7.55|650000|
| Suresh Kumar| Senior Engineer| 6.98|600000|
|     John Doe|       Developer|  6.2|550000|
|Lisa Anderson|       Developer| 6.04|540000|
| Ramesh Singh|   Data_Engineer| 5.41|500000|
+-------------+----------------+-----+------+

+---------+-----------